## 1.ライブラリのimport

In [ ]:
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import numpy as np 
from PIL import Image
import glob
import sys
plt.gray()

## 2.前処理はんだ付け箇所のトリミング

In [57]:
#基板の外枠座標を出力する関数

def Coordinate(results):
    contours = results
    x1 = [] #x座標の最小値
    y1 = [] #y座標の最小値
    x2 = [] #x座標の最大値
    y2 = [] #y座標の最大値
    for i in range(1, len(contours)):# i = 1 は画像全体の外枠になるのでカウントに入れない
        ret = cv2.boundingRect(contours[i])
        x1.append(ret[0])
        y1.append(ret[1])
        x2.append(ret[0] + ret[2])
        y2.append(ret[1] + ret[3])

    # 輪郭の一番外枠を切り抜き
    x1_min = min(x1)
    y1_min = min(y1)
    x2_max = max(x2)
    y2_max = max(y2)

    x_senter =  x1_min + (x2_max - x1_min)/2
    y_senter =  y1_min + (y2_max - y1_min)/2

    judg_x = x2_max - x1_min
    judg_y = y2_max - y1_min
    print(judg_x)
    print(judg_y)
    if (judg_x > judg_y):
        x1_min = x_senter - (judg_x/2)
        x2_max = x_senter + (judg_x/2)
        y1_min = y_senter - (judg_x/2)
        y2_max = y_senter + (judg_x/2)
    else:
        x1_min = x_senter - (judg_y/2)
        x2_max = x_senter + (judg_y/2)
        y1_min = y_senter - (judg_y/2)
        y2_max = y_senter + (judg_y/2)

    if x1_min < 0:
        x1_min = 0

    if y1_min < 0:
        y1_min = 0
    
    return(x1_min,y1_min,x2_max,y2_max)

In [62]:
#基盤の色によって半田部分のみトリミングする関数

def Trim_precessing(path):
    img = cv2.imread(path)
    hsvLower = np.array([30, 80, 0])    # 抽出する色の下限(HSV)
    hsvUpper = np.array([90, 255, 255])    # 抽出する色の上限(HSV)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # 画像をHSVに変換
    hsv_mask = cv2.inRange(hsv, hsvLower, hsvUpper)    # HSVからマスクを作成
    contours = cv2.findContours(hsv_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    #Coordinate関数に入れる
    x1_min,y1_min,x2_max,y2_max = Coordinate(contours)
    #Opening ごみ消し
    kernel = np.ones((10,10),np.uint8)
    opening = cv2.morphologyEx(hsv_mask, cv2.MORPH_OPEN, kernel)
    #Closing しろ枠作成
    kernel = np.ones((600,600),np.uint8)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    #白黒反転
    img2 = cv2.bitwise_not(closing)
    #枠線を作る
    contours = cv2.findContours(img2 , cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    img_trim = img
    img_trim = cv2.drawContours(img_trim,contours,-1,(0,0,255),1)
    img3 = cv2.fillPoly(img_trim,contours,(0,0,0))
    #再度トリミング
    hsvLower = np.array([30, 100, 0])    # 抽出する色の下限(HSV)
    hsvUpper = np.array([90, 255, 255])    # 抽出する色の上限(HSV)
    hsv = cv2.cvtColor(img3, cv2.COLOR_BGR2HSV) # 画像をHSVに変換
    hsv_mask2 = cv2.inRange(hsv, hsvLower, hsvUpper)    # HSVからマスクを作成
    #opening ごみ消し　２回目
    kernel = np.ones((10,10),np.uint8)
    opening2 = cv2.morphologyEx(hsv_mask2, cv2.MORPH_OPEN, kernel)
    #closing ごみ消し
    kernel = np.ones((60,60),np.uint8)
    closing2 = cv2.morphologyEx(opening2, cv2.MORPH_CLOSE, kernel)
    #白黒反転
    img3 = cv2.bitwise_not(closing2)
    #白を膨張させる
    kernel = np.ones((80,80),np.uint8)
    dilation = cv2.dilate(img3,kernel,iterations = 1)
    #マスクでトリミング
    img_3ch = cv2.cvtColor(dilation,cv2.COLOR_GRAY2BGR)
    img_masked = cv2.bitwise_and(img_3ch,img)
    
    crop_img = img_masked[int(y1_min):int(y2_max), int(x1_min):int(x2_max)]
    reseize_img = cv2.resize(crop_img,dsize=(224,224))
    cv2.imwrite(path,reseize_img)
    return reseize_img

## 3.画像の変更＆保存

In [ ]:
names = ["bridge","horn","potato","regular"]
path = "ファイルパス"
for name in names:
    for path in glob.glob("path" + name + "\\*jpeg"):
        Trim_precessing(path)